# SQL 的五十道練習

> 垂直與水平合併

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 在每份練習題的開始，都會先將四個學習資料庫載入環境。
- 因此 SQL 可以指定四個學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選上方選單的 Kernel -> Restart & Run All -> Restart and Run All Cells。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [1]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 35. 從 `covid19` 資料庫查詢截至 2021-03-31 全球前十大確診人數的國家，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(10, 2) 的查詢結果。

```
   Country_Region  total_confirmed
0              US         30459802
1          Brazil         12748747
2           India         12149335
3          France          4705068
4          Russia          4494234
5  United Kingdom          4359982
6           Italy          3584899
7          Turkey          3317182
8           Spain          3284353
9         Germany          2843644
```

In [2]:
find_top_ten_confirmed_country_region_from_covid19 =\
"""
-- SQL 查詢語法起點
SELECT lookup_table.Country_Region,
       SUM(daily_report.Confirmed) AS total_confirmed
  FROM daily_report
  JOIN lookup_table
    ON daily_report.Combined_Key = lookup_table.Combined_Key
 GROUP BY lookup_table.Country_Region
 ORDER BY total_confirmed DESC
 LIMIT 10;
-- SQL 查詢語法終點
"""

## 36. 從 `twElection2020` 資料庫查詢中國國民黨、民主進步黨與親民黨在不分區立委與區域立委的得票率，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(6, 3) 的查詢結果。

```
   party election  votes_percentage
0  中國國民黨    不分區立委            0.3336
1  民主進步黨    不分區立委            0.3398
2    親民黨    不分區立委            0.0366
3  中國國民黨     區域立委            0.4071
4  民主進步黨     區域立委            0.4511
5    親民黨     區域立委            0.0043
```

In [3]:
summarize_party_votes_percentages_from_twelection2020 =\
"""
-- SQL 查詢語法起點
SELECT parties.party,
       '不分區立委' AS election,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM legislative_at_large), 4) AS votes_percentage
  FROM legislative_at_large
  JOIN parties
    ON legislative_at_large.party_id = parties.id
 WHERE parties.party IN ('中國國民黨', '民主進步黨', '親民黨')
 GROUP BY legislative_at_large.party_id
 UNION
SELECT parties.party,
       '區域立委' AS election,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM legislative_regional), 4) AS votes_percentage
  FROM legislative_regional
  JOIN candidates
    ON legislative_regional.candidate_id = candidates.id
  JOIN parties
    ON candidates.party_id = parties.id
 WHERE parties.party IN ('中國國民黨', '民主進步黨', '親民黨')
 GROUP BY parties.id
 ORDER BY election;
-- SQL 查詢語法終點
"""

## 37. 從 `nba` 資料庫查詢截至 2021-03-31 洛杉磯湖人隊（Los Angeles Lakers）球員的生涯場均得分 `ppg`，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(16, 3) 的查詢結果。

```
             team_name               player_name   ppg
0   Los Angeles Lakers              LeBron James  27.0
1   Los Angeles Lakers             Anthony Davis  23.9
2   Los Angeles Lakers                Kyle Kuzma  15.4
3   Los Angeles Lakers            Andre Drummond  14.6
4   Los Angeles Lakers                Marc Gasol  14.2
5   Los Angeles Lakers           Dennis Schroder  14.2
6   Los Angeles Lakers          Montrezl Harrell  13.0
7   Los Angeles Lakers           Wesley Matthews  12.7
8   Los Angeles Lakers  Kentavious Caldwell-Pope  11.3
9   Los Angeles Lakers           Markieff Morris  11.1
10  Los Angeles Lakers       Talen Horton-Tucker   7.6
11  Los Angeles Lakers              Jared Dudley   7.3
12  Los Angeles Lakers               Alex Caruso   5.6
13  Los Angeles Lakers          Alfonzo McKinnie   4.0
14  Los Angeles Lakers            Devontae Cacok   2.2
15  Los Angeles Lakers      Kostas Antetokounmpo   1.1
```

In [4]:
list_lakers_ppg_from_nba =\
"""
-- SQL 查詢語法起點
SELECT teams.fullName AS team_name,
       players.firstName || ' ' || players.lastName AS player_name,
       career_summaries.ppg
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE teams.nickname = 'Lakers'
 ORDER BY career_summaries.ppg DESC;
-- SQL 查詢語法終點
"""

## 38. 從 `nba` 資料庫查詢各個球隊的得分王（生涯場均得分 `ppg` 全隊最高）是誰，將查詢結果依隊伍名排序，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(30, 3) 的查詢結果。

```
                      team                   player   ppg
0            Atlanta Hawks               Trae Young  24.0
1           Boston Celtics             Kemba Walker  19.8
2            Brooklyn Nets             Kevin Durant  27.1
3        Charlotte Hornets              LaMelo Ball  15.9
4            Chicago Bulls              Zach LaVine  18.8
5      Cleveland Cavaliers            Collin Sexton  19.7
6         Dallas Mavericks              Luka Doncic  25.6
7           Denver Nuggets             Nikola Jokic  18.0
8          Detroit Pistons             Josh Jackson  12.0
9    Golden State Warriors            Stephen Curry  23.8
10         Houston Rockets                John Wall  19.0
11          Indiana Pacers              T.J. Warren  15.5
12             LA Clippers              Paul George  20.1
13      Los Angeles Lakers             LeBron James  27.0
14       Memphis Grizzlies                Ja Morant  18.4
15              Miami Heat           Victor Oladipo  17.6
16         Milwaukee Bucks    Giannis Antetokounmpo  20.7
17  Minnesota Timberwolves       Karl-Anthony Towns  22.8
18    New Orleans Pelicans          Zion Williamson  25.0
19         New York Knicks             Derrick Rose  18.5
20   Oklahoma City Thunder  Shai Gilgeous-Alexander  16.3
21           Orlando Magic              Gary Harris  12.0
22      Philadelphia 76ers              Joel Embiid  24.7
23            Phoenix Suns             Devin Booker  22.8
24  Portland Trail Blazers           Damian Lillard  24.6
25        Sacramento Kings             De'Aaron Fox  17.8
26       San Antonio Spurs            DeMar DeRozan  20.1
27         Toronto Raptors               Kyle Lowry  14.8
28               Utah Jazz         Donovan Mitchell  23.2
29      Washington Wizards        Russell Westbrook  23.2
```

In [5]:
list_ppg_leader_by_teams_from_nba =\
"""
-- SQL 查詢語法起點
SELECT teams.fullName AS team,
       players.firstName || ' ' || players.lastName AS player,
       MAX(career_summaries.ppg) AS ppg
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 GROUP BY teams.fullName
 ORDER BY teams.fullName;
-- SQL 查詢語法終點
"""

## 39. 從 `imdb` 資料庫中查詢 Tom Hanks 與 Leonardo DiCaprio 在 IMDb.com 最高評價的 250 部電影中演出哪些電影，依據 `casting` 資料表中的 `ord` 衍生計算欄位 `is_lead_actor` 註記是否為第一主角（`ord` 若為 1 表示為第一主角），將查詢結果依 `release_year` 排序，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(12, 4) 的查詢結果。

```
    release_year                    title               name  is_lead_actor
0           1994             Forrest Gump          Tom Hanks              1
1           1995                Toy Story          Tom Hanks              1
2           1998      Saving Private Ryan          Tom Hanks              1
3           1999           The Green Mile          Tom Hanks              1
4           2002      Catch Me If You Can  Leonardo DiCaprio              1
5           2002      Catch Me If You Can          Tom Hanks              0
6           2006             The Departed  Leonardo DiCaprio              1
7           2010                Inception  Leonardo DiCaprio              1
8           2010              Toy Story 3          Tom Hanks              1
9           2010           Shutter Island  Leonardo DiCaprio              1
10          2012         Django Unchained  Leonardo DiCaprio              0
11          2013  The Wolf of Wall Street  Leonardo DiCaprio              1
```

In [6]:
list_movies_in_which_tom_and_leonardo_appeared =\
"""
-- SQL 查詢語法起點
SELECT movies.release_year,
       movies.title,
       actors.name,
       CASE WHEN casting.ord = 1 THEN 1
            ELSE 0 END AS is_lead_actor
  FROM actors
  JOIN casting
    ON actors.id = casting.actor_id
  JOIN movies
    ON casting.movie_id = movies.id
 WHERE actors.name IN ('Tom Hanks', 'Leonardo DiCaprio')
 ORDER BY movies.release_year;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All -> Restart and Run All Cells.

In [7]:
class TestUnionJoin(unittest.TestCase):
    def test_35_find_top_ten_confirmed_country_region_from_covid19(self):
        top_ten_confirmed_country_region_from_covid19 = pd.read_sql(find_top_ten_confirmed_country_region_from_covid19, conn)
        self.assertEqual(top_ten_confirmed_country_region_from_covid19.shape, (10, 2))
        column_values = set(top_ten_confirmed_country_region_from_covid19.iloc[:, 0].values)
        self.assertTrue('US' in column_values)
        self.assertTrue('Brazil' in column_values)
        self.assertTrue('India' in column_values)
        self.assertTrue('Turkey' in column_values)
        self.assertTrue('Spain' in column_values)
        self.assertTrue('Germany' in column_values)
    def test_36_summarize_party_votes_percentages_from_twelection2020(self):
        party_votes_percentages_from_twelection2020 = pd.read_sql(summarize_party_votes_percentages_from_twelection2020, conn)
        self.assertEqual(party_votes_percentages_from_twelection2020.shape, (6, 3))
        parties = set(party_votes_percentages_from_twelection2020.iloc[:, 0].values)
        self.assertTrue('中國國民黨' in parties)
        self.assertTrue('民主進步黨' in parties)
        self.assertTrue('親民黨' in parties)
        elections = set(party_votes_percentages_from_twelection2020.iloc[:, 1].values)
        self.assertTrue('不分區立委' in elections)
        self.assertTrue('區域立委' in elections)
    def test_37_list_lakers_ppg_from_nba(self):
        lakers_ppg_from_nba = pd.read_sql(list_lakers_ppg_from_nba, conn)
        self.assertEqual(lakers_ppg_from_nba.shape, (16, 3))
        column_values = set(lakers_ppg_from_nba.iloc[:, 0].values)
        self.assertTrue('Los Angeles Lakers' in column_values)
        column_values = set(lakers_ppg_from_nba.iloc[:, 1].values)
        self.assertTrue('LeBron James' in column_values)
        self.assertTrue('Anthony Davis' in column_values)
        self.assertTrue('Andre Drummond' in column_values)
    def test_38_list_ppg_leader_by_teams_from_nba(self):
        ppg_leader_by_teams_from_nba = pd.read_sql(list_ppg_leader_by_teams_from_nba, conn)
        self.assertEqual(ppg_leader_by_teams_from_nba.shape, (30, 3))
        teams = set(ppg_leader_by_teams_from_nba.iloc[:, 0].values)
        self.assertTrue('Boston Celtics' in teams)
        self.assertTrue('Los Angeles Lakers' in teams)
        self.assertTrue('Philadelphia 76ers' in teams)
        players = set(ppg_leader_by_teams_from_nba.iloc[:, 1].values)
        self.assertTrue('Kemba Walker' in players)
        self.assertTrue('LeBron James' in players)
        self.assertTrue('Joel Embiid' in players)
    def test_39_list_movies_in_which_tom_and_leonardo_appeared(self):
        movies_in_which_tom_and_leonardo_appeared = pd.read_sql(list_movies_in_which_tom_and_leonardo_appeared, conn)
        self.assertEqual(movies_in_which_tom_and_leonardo_appeared.shape, (12, 4))
        actors = set(movies_in_which_tom_and_leonardo_appeared.iloc[:, 2].values)
        self.assertTrue('Tom Hanks' in actors)
        self.assertTrue('Leonardo DiCaprio' in actors)
        titles = set(movies_in_which_tom_and_leonardo_appeared.iloc[:, 1].values)
        self.assertTrue('Forrest Gump' in titles)
        self.assertTrue('Saving Private Ryan' in titles)
        self.assertTrue('Catch Me If You Can' in titles)
        self.assertTrue('Inception' in titles)
        self.assertTrue('The Wolf of Wall Street' in titles)

suite = unittest.TestLoader().loadTestsFromTestCase(TestUnionJoin)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)
cwd = os.getcwd()
folder_name = cwd.split("/")[-1]
with open("../exercise_index.json", "r") as content:
    exercise_index = json.load(content)
chapter_name = exercise_index[folder_name]

test_35_summarize_party_votes_percentages (__main__.TestUnionJoin) ... ok
test_36_summarize_party_votes_percentages_from_twelection2020 (__main__.TestUnionJoin) ... ok
test_37_list_lakers_ppg_from_nba (__main__.TestUnionJoin) ... ok
test_38_list_ppg_leader_by_teams_from_nba (__main__.TestUnionJoin) ... ok
test_39_list_movies_in_which_tom_and_leonardo_appeared (__main__.TestUnionJoin) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.459s

OK


In [8]:
print("您在「{}」章節中的 {} 道 SQL 練習答對了 {} 題。".format(chapter_name, number_of_test_runs, number_of_successes))

您在「垂直與水平合併」章節中的 5 道 SQL 練習答對了 5 題。
